In [78]:
import pandas as pd
import numpy as np

In [2]:
OHCO = ['doc_source','doc_id', 'sent_num', 'token_num']
articles = OHCO[:2]
sentences = OHCO[:3]

In [70]:
TOKEN = pd.read_csv("TOKEN.csv").set_index(OHCO)
VOCAB = pd.read_csv("VOCAB.csv").set_index('term_id')
TOKEN.head()

token  pos term_str  term_id
doc_source doc_id sent_num token_num                                
PowerLine  48     1        1             I’ve  VBN      ive    85795
                           2             been  VBG     been    17768
                           3          falling   IN  falling    61167
                           4           behind   IN   behind    18021
                           5               on   JJ       on   114185

In [30]:
VOCAB = VOCAB[~VOCAB.term_str.isna()]
TOKEN = TOKEN[~TOKEN.term_str.isna()]

19585678

In [71]:
VOCAB.n.describe()

count    1.799400e+05
mean     1.097648e+02
std      4.003801e+03
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      6.000000e+00
max      1.103705e+06
Name: n, dtype: float64

In [72]:
VSHORT = VOCAB[VOCAB.n > 1]

In [73]:
len(VSHORT)


85698

In [75]:
VSHORT

,term_rank,term_str,n,stop,p_stem,pos_max
term_id,,,,,,
158064,1,the,1103705,1,the,NN
160412,2,to,558006,1,to,VB
113256,3,of,509160,1,of,DT
10479,4,and,451366,1,and,NNP
4833,5,a,439586,1,a,NN
...,...,...,...,...,...,...
7648,85694,ageila,2,0,ageila,CC
179165,85695,zolfaghar,2,0,zolfaghar,IN
9106,85696,alliesincluding,2,0,alliesinclud,NNS


In [74]:
len(TOKEN)

19751084

In [76]:
TOKEN = TOKEN[TOKEN.term_id.isin(VSHORT.index)]

In [77]:
len(TOKEN)

19656842

In [59]:
def createTFIDF(TOKEN, bag=articles, count_method='c', tf_method='raw', idf_method='standard'):
    '''Creates TFIDF table using specified method'''
    BOW = TOKEN.groupby(bag+['term_id']).term_id.count()\
    .to_frame().rename(columns={'term_id':'n'})
    BOW['c'] = BOW.n.astype('bool').astype('int')
    DTCM = BOW[count_method].unstack().fillna(0).astype('int')
    
    print('TF method:', tf_method)
    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()
    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'log':
        TF = np.log10(1 + DTCM.T)
    elif tf_method == 'raw':
        TF = DTCM.T
    elif tf_method == 'double_norm':
        TF = tf_norm_k + (1 - tf_norm_k) * TF[TF > 0] 
    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
        
    TF = TF.T
    DF = DTCM[DTCM > 0].count()
    N = DTCM.shape[0]
    
    print('IDF method:', idf_method)
    if idf_method == 'standard':
        IDF = np.log10(N / DF)
    elif idf_method == 'max':
        IDF = np.log10(DF.max() / DF) 
    elif idf_method == 'smooth':
        IDF = np.log10(N / (1 + DF)) + 1 
    elif idf_method == 'prob':  #Probabilistic method as described by wikipedia
        IDF = np.log10((N - DF) / DF)
    
    TFIDF = TF * IDF
    return TFIDF

#### Generate a few TFIDF tables using different methods to pick one

In [ ]:
createTFIDF(TOKEN)

TF method: raw
